In [11]:
import numpy as np
import scipy.stats

from model.utils.reactions import determine_shift_mu_sigma, ConstantRandomVariable
from model.sys_params import CustomDelays
from model.types.reaction_time import ReactionTime


def precompute_reaction_delays(custom_delays: CustomDelays):
    reaction_delay_params = {
            ReactionTime.NoReaction.value: ConstantRandomVariable(2**32 - 1),  # 4
            ReactionTime.Slow.value: calculate_reaction_delay_params(
                custom_delays.normal_max_delay, custom_delays.slow_max_delay
            ),  # 3
            ReactionTime.Normal.value: ConstantRandomVariable(10),
            ReactionTime.Quick.value: ConstantRandomVariable(10),
        }
    return reaction_delay_params[ReactionTime.Slow.value]
    

def calculate_reaction_delay_params(min_time, max_time, p=0.99, median_parameter=0.5):
    median_parameter_adjusted = median_parameter + (1 - median_parameter) * min_time / max_time
    shift, median, sigma = determine_shift_mu_sigma(
        left_bound=0, right_bound=max_time, p=p, median_parameter=median_parameter_adjusted
    )
    return sigma, shift, median


In [12]:
custom_delays = [
    CustomDelays(slow_max_delay=3600 * 24 * 15),
    CustomDelays(slow_max_delay=3600 * 24 * 30),
    CustomDelays(slow_max_delay=3600 * 24 * 45),
    CustomDelays(slow_max_delay=3600 * 24 * 60),
]

for custom_delay in custom_delays:
    print(precompute_reaction_delays(custom_delay))

(np.float64(0.1742925521297363), 0, 864000.0)
(np.float64(0.2316921328693872), 0, 1512000.0)
(np.float64(0.25266499110519525), 0, 2160000.0)
(np.float64(0.263548061632525), 0, 2808000.0)
